In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [25]:
def get_facebook_insights(page_id, access_token, metric, start_date, end_date):
    url = f'https://graph.facebook.com/v20.0/{page_id}/insights'
    
    # Convert start and end date to datetime objects
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    
    all_data = []
    
    while start <= end:
        # Calculate the end date for this iteration (30 days or less)
        iteration_end = min(start + timedelta(days=30), end)
        
        params = {
            'access_token': access_token,
            'metric': metric,
            'since': start.strftime('%Y-%m-%d'),
            'until': iteration_end.strftime('%Y-%m-%d')
        }
        
        print(f"Making API call for period: {start.strftime('%Y-%m-%d')} to {iteration_end.strftime('%Y-%m-%d')}")
        
        response = requests.get(url, params=params)
        
        if response.status_code != 200:
            print(f"Error in API call: {response.status_code}")
            print(f"Response content: {response.text}")
            return pd.DataFrame()  # Return empty DataFrame if an error occurs
        
        data = response.json()
        
        if 'data' in data and data['data']:
            # Extend the list with the values from this API call
            all_data.extend(data['data'][0]['values'])
        else:
            print(f"No data returned for period: {start.strftime('%Y-%m-%d')} to {iteration_end.strftime('%Y-%m-%d')}")
        
        # Move the start date to the day after the current iteration's end date
        start = iteration_end + timedelta(days=1)
    
    # If no data was collected, return an empty DataFrame
    if not all_data:
        print("No data collected across all API calls")
        return pd.DataFrame()
    
    # Create DataFrame and ensure 'end_time' is a datetime column
    df = pd.DataFrame(all_data)
    if 'end_time' in df.columns:
        df['end_time'] = pd.to_datetime(df['end_time'])
    
    return df

In [32]:
page_id = '151189004972368'
access_token = 'EAAPGyUspGX4BO1tTxi02gxCBOWq1d6d7RHBbs3fwr7NzZAn9lRZCLSjUCKZBu4QonZBn9ZAZA5xRd26cVqECC4A65QawLozpuwM99rDcKr42E0JZAoDV9Ww2GSFz1dcaCqlkVNv8ERtiyQr8T8fxoCgLTyxHfYBVZBH8umwetQiO7iRWftuX3BtaYfMwf6AUNGmhPq56C56zsvMaMZClSYcQ5bL61DF3wLJYZD'
start_date = '2023-09-01'
end_date = '2024-09-12'
metric = 'page_follows'

In [ ]:
df = get_facebook_insights(page_id, access_token, metric, start_date, end_date)
print(df)

In [ ]:
df